In [ ]:
%%bash
cd /mnt/tmp
wget s3.amazonaws.com/geowave/latest/scripts/emr/quickstart/geowave-env.sh
source /mnt/tmp/geowave-env.sh
mkdir gdelt
cd gdelt
wget http://data.gdeltproject.org/events/md5sums
for file in `cat md5sums | cut -d' ' -f3 | grep "^${TIME_REGEX}"` ; \
do wget http://data.gdeltproject.org/events/$file ; done
md5sum -c md5sums 2>&1 | grep "^${TIME_REGEX}"

In [ ]:
%%bash

# We have to source here again because bash runs in a separate sub process each cell.
source /mnt/tmp/geowave-env.sh

# clear old potential runs
geowave config rmstore gdelt
geowave config rmindex gdelt-spatial

geowave config addstore gdelt --gwNamespace geowave.gdelt -t accumulo --zookeeper $HOSTNAME:2181 --instance accumulo --user root --password secret

# configure a spatial index
geowave config addindex -t spatial gdelt-spatial --partitionStrategy round_robin --numPartitions $NUM_PARTITIONS

# run the ingest for a 10x10 deg bounding box over Europe
geowave ingest localtogw /mnt/tmp/gdelt gdelt gdelt-spatial -f gdelt \
--gdelt.cql "BBOX(geometry, 0, 50, 10, 60)"

In [ ]:
accumulo_store = accumulo_options_class()
accumulo_store.setInstance('accumulo')
accumulo_store.setUser('root')
accumulo_store.setPassword('secret')
accumulo_store.setZookeeper(os.environ['HOSTNAME'] + ':2181')
accumulo_store.setGeowaveNamespace('geowave.gdelt')


accumulo_store_options = accumulo_store.createPluginOptions()

In [ ]:
adapter_id = byte_array_class('gdeltevent')
adapter = accumulo_store_options.createAdapterStore().getAdapter(adapter_id)

rdd_opts = rdd_options_class()
rdd_opts.setQueryOptions(query_options_class(adapter))
rdd_opts.setMinSplits(1000)
rdd_opts.setMaxSplits(1000)

geowave_rdd = rdd_loader_class.loadRDD(sc._jsc.sc(), accumulo_store_options, rdd_opts)